In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, max, count, collect_list, concat_ws, coalesce, upper
import os

# Configure Spark for Windows compatibility
spark = SparkSession.builder \
    .appName("NutritionalValuesGenerator") \
    .config("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.speculation", "false") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.cleanup-failures.ignored", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [3]:
dataset_dir = "D:/1-master/2024-2026/1-ISST/BigDataAnalitics/1-labs/fii-bda/FII-BDA/sampled_dataset"


df_food = spark.read.parquet(f"{dataset_dir}/food.parquet")
df_food_nutrient = spark.read.parquet(f"{dataset_dir}/food_nutrient.parquet")
df_nutrient = spark.read.parquet(f"{dataset_dir}/nutrient.parquet")
df_food_portion = spark.read.parquet(f"{dataset_dir}/food_portion.parquet")
df_measure_unit = spark.read.parquet(f"{dataset_dir}/measure_unit.parquet")


print(f"Food records: {df_food.count()}")
print(f"Food-Nutrient records: {df_food_nutrient.count()}")
print(f"Nutrient records: {df_nutrient.count()}")
print(f"Food Portion records: {df_food_portion.count()}")
print(f"Measure Unit records: {df_measure_unit.count()}")

Food records: 5000
Food-Nutrient records: 65141
Nutrient records: 477
Food Portion records: 104
Measure Unit records: 5


In [4]:
df_food_nutrient_clean = df_food_nutrient.select(
    col("fdc_id"),
    col("nutrient_id"),
    col("amount"),
    col("data_points"),
    col("min"),
    col("max"),
    col("median")
).filter(col("amount").isNotNull())

df_nutrient_clean = df_nutrient.select(
    col("id").alias("nutrient_id"),
    col("name").alias("nutrient_name"),
    col("unit_name").alias("nutrient_unit"),
    col("rank")
)

df_food_enriched = df_food.select(
    col("fdc_id"),
    col("description"),
    col("data_type")
)

df_merged = df_food_enriched.join(
    df_food_nutrient_clean,
    on="fdc_id",
    how="inner"
).join(
    df_nutrient_clean,
    on="nutrient_id",
    how="left"
)

df_merged = df_merged.fillna("Unknown", subset=["nutrient_name", "nutrient_unit"])

df_merged = df_merged.withColumn(
    "nutrient_display",
    when(col("nutrient_unit").isNotNull(),
         concat_ws(" (", col("nutrient_name"), concat_ws(")", col("nutrient_unit")))
    ).otherwise(col("nutrient_name"))
)

print(f"Merged records: {df_merged.count()}")

Merged records: 65141


In [5]:
df_aggregated = df_merged.groupBy("fdc_id", "description", "data_type").agg(
    collect_list("nutrient_display").alias("nutrients"),
    collect_list("amount").alias("amounts"),
    collect_list("median").alias("medians"),
    collect_list("min").alias("mins"),
    collect_list("max").alias("maxs"),
    count("nutrient_id").alias("total_nutrients")
)

df_aggregated = df_aggregated.withColumn(
    "nutrients_json",
    concat_ws("|", col("nutrients"))
)

df_pivot = df_merged.groupBy("fdc_id").pivot("nutrient_name").agg(
    coalesce(
        max(when(col("nutrient_name").isNotNull(), col("amount"))),
        max(when(col("nutrient_name").isNotNull(), col("median")))
    ).alias("amount")
)

print(f"Aggregated foods: {df_aggregated.count()}")
print(f"Pivot table columns: {len(df_pivot.columns)}")
print(f"Foods with nutrients: {df_pivot.count()}")

Aggregated foods: 4688
Pivot table columns: 180
Foods with nutrients: 4688


In [6]:
df_merged_clean = df_merged.filter(col("amount").isNotNull())

# Macronutrients
energy_col = df_merged_clean.filter(col("nutrient_name") == "Energy").select("fdc_id", col("amount").alias("energy")).dropDuplicates(["fdc_id"])
protein_col = df_merged_clean.filter(upper(col("nutrient_name")).contains("PROTEIN")).select("fdc_id", col("amount").alias("protein")).dropDuplicates(["fdc_id"])
carbs_col = df_merged_clean.filter(col("nutrient_name") == "Carbohydrate, by difference").select("fdc_id", col("amount").alias("carbs")).dropDuplicates(["fdc_id"])
total_fat_col = df_merged_clean.filter(col("nutrient_name") == "Total lipid (fat)").select("fdc_id", col("amount").alias("total_fat")).dropDuplicates(["fdc_id"])

# Food composition basics
water_col = df_merged_clean.filter(col("nutrient_name") == "Water").select("fdc_id", col("amount").alias("water")).dropDuplicates(["fdc_id"])
ash_col = df_merged_clean.filter(col("nutrient_name") == "Ash").select("fdc_id", col("amount").alias("ash")).dropDuplicates(["fdc_id"])
alcohol_col = df_merged_clean.filter(col("nutrient_name") == "Alcohol, ethyl").select("fdc_id", col("amount").alias("alcohol")).dropDuplicates(["fdc_id"])
caffeine_col = df_merged_clean.filter(col("nutrient_name") == "Caffeine").select("fdc_id", col("amount").alias("caffeine")).dropDuplicates(["fdc_id"])

# Fiber and Sugars
fiber_col = df_merged_clean.filter(col("nutrient_name").contains("Fiber, total dietary")).select("fdc_id", col("amount").alias("fiber")).dropDuplicates(["fdc_id"])
sugars_col = df_merged_clean.filter(col("nutrient_name") == "Total Sugars").select("fdc_id", col("amount").alias("sugars")).dropDuplicates(["fdc_id"])

# Sugar types
glucose_col = df_merged_clean.filter(col("nutrient_name") == "Glucose").select("fdc_id", col("amount").alias("glucose")).dropDuplicates(["fdc_id"])
fructose_col = df_merged_clean.filter(col("nutrient_name") == "Fructose").select("fdc_id", col("amount").alias("fructose")).dropDuplicates(["fdc_id"])
sucrose_col = df_merged_clean.filter(col("nutrient_name") == "Sucrose").select("fdc_id", col("amount").alias("sucrose")).dropDuplicates(["fdc_id"])
lactose_col = df_merged_clean.filter(col("nutrient_name") == "Lactose").select("fdc_id", col("amount").alias("lactose")).dropDuplicates(["fdc_id"])

# Fats breakdown
saturated_fat_col = df_merged_clean.filter(col("nutrient_name") == "Fatty acids, total saturated").select("fdc_id", col("amount").alias("saturated_fat")).dropDuplicates(["fdc_id"])
monounsaturated_fat_col = df_merged_clean.filter(col("nutrient_name") == "Fatty acids, total monounsaturated").select("fdc_id", col("amount").alias("monounsaturated_fat")).dropDuplicates(["fdc_id"])
polyunsaturated_fat_col = df_merged_clean.filter(col("nutrient_name") == "Fatty acids, total polyunsaturated").select("fdc_id", col("amount").alias("polyunsaturated_fat")).dropDuplicates(["fdc_id"])
trans_fat_col = df_merged_clean.filter(col("nutrient_name") == "Fatty acids, total trans").select("fdc_id", col("amount").alias("trans_fat")).dropDuplicates(["fdc_id"])
cholesterol_col = df_merged_clean.filter(col("nutrient_name") == "Cholesterol").select("fdc_id", col("amount").alias("cholesterol")).dropDuplicates(["fdc_id"])

# Vitamins
vitamin_a_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin A, RAE")).select("fdc_id", col("amount").alias("vitamin_a")).dropDuplicates(["fdc_id"])
vitamin_c_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin C")).select("fdc_id", col("amount").alias("vitamin_c")).dropDuplicates(["fdc_id"])
vitamin_d_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin D (D2 + D3)")).select("fdc_id", col("amount").alias("vitamin_d")).dropDuplicates(["fdc_id"])
vitamin_e_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin E (alpha-tocopherol)")).select("fdc_id", col("amount").alias("vitamin_e")).dropDuplicates(["fdc_id"])
vitamin_k_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin K (phylloquinone)")).select("fdc_id", col("amount").alias("vitamin_k")).dropDuplicates(["fdc_id"])

# B Vitamins
thiamin_col = df_merged_clean.filter(col("nutrient_name").contains("Thiamin")).select("fdc_id", col("amount").alias("thiamin_b1")).dropDuplicates(["fdc_id"])
riboflavin_col = df_merged_clean.filter(col("nutrient_name").contains("Riboflavin")).select("fdc_id", col("amount").alias("riboflavin_b2")).dropDuplicates(["fdc_id"])
niacin_col = df_merged_clean.filter(col("nutrient_name").contains("Niacin")).select("fdc_id", col("amount").alias("niacin_b3")).dropDuplicates(["fdc_id"])
vitamin_b6_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin B-6")).select("fdc_id", col("amount").alias("vitamin_b6")).dropDuplicates(["fdc_id"])
folate_col = df_merged_clean.filter(col("nutrient_name") == "Folate, total").select("fdc_id", col("amount").alias("folate")).dropDuplicates(["fdc_id"])
vitamin_b12_col = df_merged_clean.filter(col("nutrient_name").contains("Vitamin B-12")).select("fdc_id", col("amount").alias("vitamin_b12")).dropDuplicates(["fdc_id"])

# Additional micronutrients
choline_col = df_merged_clean.filter(col("nutrient_name").contains("Choline, total")).select("fdc_id", col("amount").alias("choline")).dropDuplicates(["fdc_id"])

# Minerals
calcium_col = df_merged_clean.filter(col("nutrient_name") == "Calcium, Ca").select("fdc_id", col("amount").alias("calcium")).dropDuplicates(["fdc_id"])
iron_col = df_merged_clean.filter(col("nutrient_name") == "Iron, Fe").select("fdc_id", col("amount").alias("iron")).dropDuplicates(["fdc_id"])
magnesium_col = df_merged_clean.filter(col("nutrient_name") == "Magnesium, Mg").select("fdc_id", col("amount").alias("magnesium")).dropDuplicates(["fdc_id"])
phosphorus_col = df_merged_clean.filter(col("nutrient_name") == "Phosphorus, P").select("fdc_id", col("amount").alias("phosphorus")).dropDuplicates(["fdc_id"])
potassium_col = df_merged_clean.filter(col("nutrient_name") == "Potassium, K").select("fdc_id", col("amount").alias("potassium")).dropDuplicates(["fdc_id"])
sodium_col = df_merged_clean.filter(col("nutrient_name") == "Sodium, Na").select("fdc_id", col("amount").alias("sodium")).dropDuplicates(["fdc_id"])
zinc_col = df_merged_clean.filter(col("nutrient_name") == "Zinc, Zn").select("fdc_id", col("amount").alias("zinc")).dropDuplicates(["fdc_id"])
copper_col = df_merged_clean.filter(col("nutrient_name") == "Copper, Cu").select("fdc_id", col("amount").alias("copper")).dropDuplicates(["fdc_id"])
manganese_col = df_merged_clean.filter(col("nutrient_name") == "Manganese, Mn").select("fdc_id", col("amount").alias("manganese")).dropDuplicates(["fdc_id"])
selenium_col = df_merged_clean.filter(col("nutrient_name") == "Selenium, Se").select("fdc_id", col("amount").alias("selenium")).dropDuplicates(["fdc_id"])

# Carotenoids and antioxidants
beta_carotene_col = df_merged_clean.filter(col("nutrient_name").contains("Carotene, beta")).select("fdc_id", col("amount").alias("beta_carotene")).dropDuplicates(["fdc_id"])
lycopene_col = df_merged_clean.filter(col("nutrient_name") == "Lycopene").select("fdc_id", col("amount").alias("lycopene")).dropDuplicates(["fdc_id"])
lutein_col = df_merged_clean.filter(col("nutrient_name").contains("Lutein + zeaxanthin")).select("fdc_id", col("amount").alias("lutein_zeaxanthin")).dropDuplicates(["fdc_id"])

# Join all nutrients
df_metrics = df_food_enriched.join(energy_col, on="fdc_id", how="left") \
    .join(protein_col, on="fdc_id", how="left") \
    .join(carbs_col, on="fdc_id", how="left") \
    .join(total_fat_col, on="fdc_id", how="left") \
    .join(water_col, on="fdc_id", how="left") \
    .join(ash_col, on="fdc_id", how="left") \
    .join(alcohol_col, on="fdc_id", how="left") \
    .join(caffeine_col, on="fdc_id", how="left") \
    .join(fiber_col, on="fdc_id", how="left") \
    .join(sugars_col, on="fdc_id", how="left") \
    .join(glucose_col, on="fdc_id", how="left") \
    .join(fructose_col, on="fdc_id", how="left") \
    .join(sucrose_col, on="fdc_id", how="left") \
    .join(lactose_col, on="fdc_id", how="left") \
    .join(saturated_fat_col, on="fdc_id", how="left") \
    .join(monounsaturated_fat_col, on="fdc_id", how="left") \
    .join(polyunsaturated_fat_col, on="fdc_id", how="left") \
    .join(trans_fat_col, on="fdc_id", how="left") \
    .join(cholesterol_col, on="fdc_id", how="left") \
    .join(vitamin_a_col, on="fdc_id", how="left") \
    .join(vitamin_c_col, on="fdc_id", how="left") \
    .join(vitamin_d_col, on="fdc_id", how="left") \
    .join(vitamin_e_col, on="fdc_id", how="left") \
    .join(vitamin_k_col, on="fdc_id", how="left") \
    .join(thiamin_col, on="fdc_id", how="left") \
    .join(riboflavin_col, on="fdc_id", how="left") \
    .join(niacin_col, on="fdc_id", how="left") \
    .join(vitamin_b6_col, on="fdc_id", how="left") \
    .join(folate_col, on="fdc_id", how="left") \
    .join(vitamin_b12_col, on="fdc_id", how="left") \
    .join(choline_col, on="fdc_id", how="left") \
    .join(calcium_col, on="fdc_id", how="left") \
    .join(iron_col, on="fdc_id", how="left") \
    .join(magnesium_col, on="fdc_id", how="left") \
    .join(phosphorus_col, on="fdc_id", how="left") \
    .join(potassium_col, on="fdc_id", how="left") \
    .join(sodium_col, on="fdc_id", how="left") \
    .join(zinc_col, on="fdc_id", how="left") \
    .join(copper_col, on="fdc_id", how="left") \
    .join(manganese_col, on="fdc_id", how="left") \
    .join(selenium_col, on="fdc_id", how="left") \
    .join(beta_carotene_col, on="fdc_id", how="left") \
    .join(lycopene_col, on="fdc_id", how="left") \
    .join(lutein_col, on="fdc_id", how="left")

print(f"Foods with energy: {df_metrics.filter(col('energy').isNotNull()).count()}")
print(f"Foods with protein: {df_metrics.filter(col('protein').isNotNull()).count()}")
print(f"Foods with carbs: {df_metrics.filter(col('carbs').isNotNull()).count()}")
print(f"Foods with total fat: {df_metrics.filter(col('total_fat').isNotNull()).count()}")
print(f"Foods with water: {df_metrics.filter(col('water').isNotNull()).count()}")
print(f"Foods with fiber: {df_metrics.filter(col('fiber').isNotNull()).count()}")
print(f"Foods with vitamin C: {df_metrics.filter(col('vitamin_c').isNotNull()).count()}")
print(f"Foods with calcium: {df_metrics.filter(col('calcium').isNotNull()).count()}")
print(f"Foods with iron: {df_metrics.filter(col('iron').isNotNull()).count()}")

Foods with energy: 4485
Foods with protein: 4550
Foods with carbs: 4513
Foods with total fat: 4505
Foods with water: 50
Foods with fiber: 3827
Foods with vitamin C: 2136
Foods with calcium: 3741
Foods with iron: 3747


In [7]:
df_final_profile = df_aggregated.select(
    col("fdc_id"),
    col("description").alias("food_description"),
    col("data_type").alias("food_type"),
    col("total_nutrients")
).join(
    df_metrics.select(
        "fdc_id", "energy", "protein", "carbs", "total_fat", 
        "water", "ash", "alcohol", "caffeine",
        "fiber", "sugars", "glucose", "fructose", "sucrose", "lactose",
        "saturated_fat", "monounsaturated_fat", "polyunsaturated_fat", "trans_fat", "cholesterol",
        "vitamin_a", "vitamin_c", "vitamin_d", "vitamin_e", "vitamin_k",
        "thiamin_b1", "riboflavin_b2", "niacin_b3", "vitamin_b6", "folate", "vitamin_b12",
        "choline",
        "calcium", "iron", "magnesium", "phosphorus", "potassium", "sodium", 
        "zinc", "copper", "manganese", "selenium",
        "beta_carotene", "lycopene", "lutein_zeaxanthin"
    ),
    on="fdc_id",
    how="left"
)

df_final_profile = df_final_profile.select(
    "fdc_id",
    "food_description",
    "food_type",
    "total_nutrients",
    # Macronutrients
    "energy",
    "protein",
    "carbs",
    "total_fat",
    # Food composition
    "water",
    "ash",
    "alcohol",
    "caffeine",
    # Fiber and Sugars
    "fiber",
    "sugars",
    "glucose",
    "fructose",
    "sucrose",
    "lactose",
    # Fats breakdown
    "saturated_fat",
    "monounsaturated_fat",
    "polyunsaturated_fat",
    "trans_fat",
    "cholesterol",
    # Vitamins
    "vitamin_a",
    "vitamin_c",
    "vitamin_d",
    "vitamin_e",
    "vitamin_k",
    # B Vitamins
    "thiamin_b1",
    "riboflavin_b2",
    "niacin_b3",
    "vitamin_b6",
    "folate",
    "vitamin_b12",
    # Additional nutrients
    "choline",
    # Minerals
    "calcium",
    "iron",
    "magnesium",
    "phosphorus",
    "potassium",
    "sodium",
    "zinc",
    "copper",
    "manganese",
    "selenium",
    # Carotenoids
    "beta_carotene",
    "lycopene",
    "lutein_zeaxanthin"
)

print(f"Final nutritional profiles: {df_final_profile.count()}")
print(f"Total columns: {len(df_final_profile.columns)}")
print(f"\nSample of comprehensive nutritional data:")
df_final_profile.show(5, truncate=False)

Final nutritional profiles: 4688
Total columns: 48

Sample of comprehensive nutritional data:
+------+-------------------------------------+------------+---------------+------+-------+-----+---------+-----+----+-------+--------+-----+------+-------+--------+-------+-------+-------------+-------------------+-------------------+---------+-----------+---------+---------+---------+---------+---------+----------+-------------+---------+----------+------+-----------+-------+-------+----+---------+----------+---------+------+----+------+---------+--------+-------------+--------+-----------------+
|fdc_id|food_description                     |food_type   |total_nutrients|energy|protein|carbs|total_fat|water|ash |alcohol|caffeine|fiber|sugars|glucose|fructose|sucrose|lactose|saturated_fat|monounsaturated_fat|polyunsaturated_fat|trans_fat|cholesterol|vitamin_a|vitamin_c|vitamin_d|vitamin_e|vitamin_k|thiamin_b1|riboflavin_b2|niacin_b3|vitamin_b6|folate|vitamin_b12|choline|calcium|iron|magnesium|p

In [9]:
import os
from pathlib import Path

# Use Windows-compatible path (same directory level as sampled_dataset)
output_dir = "D:/1-master/2024-2026/1-ISST/BigDataAnalitics/1-labs/fii-bda/FII-BDA/output"
output_path = f"{output_dir}/nutritional_profiles"
os.makedirs(output_dir, exist_ok=True)

# Set Hadoop system properties for Windows compatibility (works without restarting kernel)
spark.sparkContext._jsc.hadoopConfiguration().set(
    "mapreduce.fileoutputcommitter.algorithm.version", "2"
)
spark.sparkContext._jsc.hadoopConfiguration().set(
    "mapreduce.fileoutputcommitter.cleanup-failures.ignored", "true"
)

# Try Spark write first, fallback to pandas/pyarrow if it fails
try:
    df_final_profile.repartition(1).write.mode("overwrite").parquet(output_path)
    print(f"✅ Results exported to {output_path} using Spark")
except Exception as e:
    print(f"⚠️ Spark write failed (Windows native IO issue): {str(e)[:200]}...")
    print("Falling back to pandas/pyarrow write...")
    
    # Convert to pandas and write with pyarrow
    import pandas as pd
    
    try:
        # Ensure output directory exists
        Path(output_path).mkdir(parents=True, exist_ok=True)
        
        # Convert Spark DataFrame to pandas
        print("Converting Spark DataFrame to pandas...")
        pdf = df_final_profile.toPandas()
        print(f"Converted {len(pdf)} rows, {len(pdf.columns)} columns")
        
        # Write to parquet file
        output_file = f"{output_path}/nutritional_profiles.parquet"
        print(f"Writing to {output_file}...")
        pdf.to_parquet(
            output_file,
            engine="pyarrow",
            index=False
        )
        
        # Verify file was created
        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024 * 1024)  # MB
            print(f"✅ Results exported successfully!")
            print(f"   File: {output_file}")
            print(f"   Size: {file_size:.2f} MB")
            print(f"   Rows: {len(pdf):,}")
            print(f"   Columns: {len(pdf.columns)}")
        else:
            print(f"❌ Error: File was not created at {output_file}")
            
    except Exception as e2:
        print(f"❌ Pandas/pyarrow write also failed: {e2}")
        import traceback
        traceback.print_exc()

print("\nGenerated files:")
if os.path.exists(output_path):
    files = [f for f in os.listdir(output_path) if os.path.isfile(os.path.join(output_path, f))]
    if files:
        for f in files:
            print(f"  - {f}")
    else:
        print("  (directory exists but no files found)")
else:
    print(f"  (directory {output_path} does not exist)")

⚠️ Spark write failed (Windows native IO issue): An error occurred while calling o1556.parquet.
: org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/D:/1-master/2024-2026/1-ISST/BigDataAnalitics/1-labs/fii-b...
Falling back to pandas/pyarrow write...
Converting Spark DataFrame to pandas...
Converted 4688 rows, 48 columns
Writing to D:/1-master/2024-2026/1-ISST/BigDataAnalitics/1-labs/fii-bda/FII-BDA/output/nutritional_profiles/nutritional_profiles.parquet...
✅ Results exported successfully!
   File: D:/1-master/2024-2026/1-ISST/BigDataAnalitics/1-labs/fii-bda/FII-BDA/output/nutritional_profiles/nutritional_profiles.parquet
   Size: 0.30 MB
   Rows: 4,688
   Columns: 48

Generated files:
  - nutritional_profiles.parquet


## 7. Food Statistics: Top Nutritious Foods


In [8]:
from pyspark.sql.functions import (
    col, when, lit, coalesce, desc, round as spark_round, concat_ws
)

# Calculate nutrition density score
# Score = (protein + fiber + vitamins + minerals) / energy
# Higher score = more nutrients per calorie

df_nutrition_score = df_final_profile.withColumn(
    "protein_score",
    when(col("energy").isNotNull() & (col("energy") > 0) & col("protein").isNotNull(),
         col("protein") / col("energy") * 100
    ).otherwise(0.0)
).withColumn(
    "fiber_score",
    when(col("energy").isNotNull() & (col("energy") > 0) & col("fiber").isNotNull(),
         col("fiber") / col("energy") * 100
    ).otherwise(0.0)
).withColumn(
    "vitamin_score",
    when(col("energy").isNotNull() & (col("energy") > 0),
         coalesce(col("vitamin_a"), lit(0)) / 900.0 +  # RDA-based normalization
         coalesce(col("vitamin_c"), lit(0)) / 90.0 +
         coalesce(col("vitamin_e"), lit(0)) / 15.0 +
         coalesce(col("vitamin_k"), lit(0)) / 120.0 +
         coalesce(col("thiamin_b1"), lit(0)) / 1.2 +
         coalesce(col("riboflavin_b2"), lit(0)) / 1.3 +
         coalesce(col("niacin_b3"), lit(0)) / 16.0 +
         coalesce(col("vitamin_b6"), lit(0)) / 1.7 +
         coalesce(col("folate"), lit(0)) / 400.0 +
         coalesce(col("vitamin_b12"), lit(0)) / 2.4
    ).otherwise(0.0)
).withColumn(
    "mineral_score",
    when(col("energy").isNotNull() & (col("energy") > 0),
         coalesce(col("calcium"), lit(0)) / 1000.0 +
         coalesce(col("iron"), lit(0)) / 18.0 +
         coalesce(col("magnesium"), lit(0)) / 400.0 +
         coalesce(col("phosphorus"), lit(0)) / 700.0 +
         coalesce(col("potassium"), lit(0)) / 4700.0 +
         coalesce(col("zinc"), lit(0)) / 11.0 +
         coalesce(col("copper"), lit(0)) / 0.9 +
         coalesce(col("manganese"), lit(0)) / 2.3 +
         coalesce(col("selenium"), lit(0)) / 55.0
    ).otherwise(0.0)
).withColumn(
    "nutrition_density_score",
    when(col("energy").isNotNull() & (col("energy") > 0),
         col("protein_score") * 0.3 +
         col("fiber_score") * 0.2 +
         col("vitamin_score") * 0.25 +
         col("mineral_score") * 0.25
    ).otherwise(0.0)
).filter(
    col("energy").isNotNull() & 
    (col("energy") > 0) &
    col("food_description").isNotNull()
)

# Get top 10 most nutritious foods
top_10_nutritious = df_nutrition_score.select(
    "fdc_id",
    "food_description",
    "food_type",
    spark_round(col("energy"), 1).alias("energy_kcal"),
    spark_round(col("protein"), 2).alias("protein_g"),
    spark_round(col("fiber"), 2).alias("fiber_g"),
    spark_round(col("vitamin_c"), 2).alias("vitamin_c_mg"),
    spark_round(col("calcium"), 2).alias("calcium_mg"),
    spark_round(col("iron"), 2).alias("iron_mg"),
    spark_round(col("nutrition_density_score"), 3).alias("nutrition_score")
).orderBy(desc("nutrition_density_score")).limit(10)

print("=" * 80)
print("TOP 10 MOST NUTRITIOUS FOODS (by nutrition density score)")
print("=" * 80)
top_10_nutritious.show(10, truncate=False)


TOP 10 MOST NUTRITIOUS FOODS (by nutrition density score)
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------+---------+-------+------------+----------+-------+---------------+
|fdc_id |food_description                                                                                                                                               |food_type   |energy_kcal|protein_g|fiber_g|vitamin_c_mg|calcium_mg|iron_mg|nutrition_score|
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------+---------+-------+------------+----------+-------+---------------+
|1705714|ORGANIC DULSE FLAKES                                                                                                                                  

## 8. Food Statistics: Top Nutrient Foods Per Portion


In [10]:
# Calculate nutrients per portion using food_portion data
# food_portion contains gram_weight which represents the weight of one portion

df_food_portion_clean = df_food_portion.select(
    col("fdc_id"),
    col("amount").alias("portion_amount"),
    col("gram_weight").alias("portion_grams"),
    col("measure_unit_id")
).filter(
    col("gram_weight").isNotNull() & 
    (col("gram_weight") > 0) &
    col("fdc_id").isNotNull()
)

# Join with measure_unit to get portion description
df_measure_unit_clean = df_measure_unit.select(
    col("id").alias("measure_unit_id"),
    col("name").alias("measure_unit_name")
)

df_portion_with_unit = df_food_portion_clean.join(
    df_measure_unit_clean,
    on="measure_unit_id",
    how="left"
).withColumn(
    "portion_description",
    when(col("measure_unit_name").isNotNull(),
         concat_ws(" ", col("portion_amount"), col("measure_unit_name"))
    ).otherwise(concat_ws(" ", col("portion_amount"), lit("units")))
)

# Join nutritional profiles with portion data
df_nutrients_per_portion = df_final_profile.join(
    df_portion_with_unit,
    on="fdc_id",
    how="inner"
).withColumn(
    # Calculate nutrients per portion (nutrients per 100g * portion_grams / 100)
    "protein_per_portion",
    when(col("protein").isNotNull() & col("portion_grams").isNotNull(),
         col("protein") * col("portion_grams") / 100.0
    ).otherwise(None)
).withColumn(
    "fiber_per_portion",
    when(col("fiber").isNotNull() & col("portion_grams").isNotNull(),
         col("fiber") * col("portion_grams") / 100.0
    ).otherwise(None)
).withColumn(
    "vitamin_c_per_portion",
    when(col("vitamin_c").isNotNull() & col("portion_grams").isNotNull(),
         col("vitamin_c") * col("portion_grams") / 100.0
    ).otherwise(None)
).withColumn(
    "calcium_per_portion",
    when(col("calcium").isNotNull() & col("portion_grams").isNotNull(),
         col("calcium") * col("portion_grams") / 100.0
    ).otherwise(None)
).withColumn(
    "iron_per_portion",
    when(col("iron").isNotNull() & col("portion_grams").isNotNull(),
         col("iron") * col("portion_grams") / 100.0
    ).otherwise(None)
).withColumn(
    "energy_per_portion",
    when(col("energy").isNotNull() & col("portion_grams").isNotNull(),
         col("energy") * col("portion_grams") / 100.0
    ).otherwise(None)
)

# Get top 5 foods per key nutrient per portion
print("=" * 80)
print("TOP 5 FOODS BY PROTEIN PER PORTION")
print("=" * 80)
top_5_protein = df_nutrients_per_portion.select(
    "food_description",
    "portion_description",
    spark_round(col("portion_grams"), 1).alias("portion_grams"),
    spark_round(col("protein_per_portion"), 2).alias("protein_per_portion_g"),
    spark_round(col("energy_per_portion"), 1).alias("energy_per_portion_kcal")
).filter(
    col("protein_per_portion").isNotNull()
).orderBy(desc("protein_per_portion")).limit(5)
top_5_protein.show(5, truncate=False)

print("\n" + "=" * 80)
print("TOP 5 FOODS BY FIBER PER PORTION")
print("=" * 80)
top_5_fiber = df_nutrients_per_portion.select(
    "food_description",
    "portion_description",
    spark_round(col("portion_grams"), 1).alias("portion_grams"),
    spark_round(col("fiber_per_portion"), 2).alias("fiber_per_portion_g"),
    spark_round(col("energy_per_portion"), 1).alias("energy_per_portion_kcal")
).filter(
    col("fiber_per_portion").isNotNull()
).orderBy(desc("fiber_per_portion")).limit(5)
top_5_fiber.show(5, truncate=False)

print("\n" + "=" * 80)
print("TOP 5 FOODS BY VITAMIN C PER PORTION")
print("=" * 80)
top_5_vitamin_c = df_nutrients_per_portion.select(
    "food_description",
    "portion_description",
    spark_round(col("portion_grams"), 1).alias("portion_grams"),
    spark_round(col("vitamin_c_per_portion"), 2).alias("vitamin_c_per_portion_mg"),
    spark_round(col("energy_per_portion"), 1).alias("energy_per_portion_kcal")
).filter(
    col("vitamin_c_per_portion").isNotNull()
).orderBy(desc("vitamin_c_per_portion")).limit(5)
top_5_vitamin_c.show(5, truncate=False)

print("\n" + "=" * 80)
print("TOP 5 FOODS BY CALCIUM PER PORTION")
print("=" * 80)
top_5_calcium = df_nutrients_per_portion.select(
    "food_description",
    "portion_description",
    spark_round(col("portion_grams"), 1).alias("portion_grams"),
    spark_round(col("calcium_per_portion"), 2).alias("calcium_per_portion_mg"),
    spark_round(col("energy_per_portion"), 1).alias("energy_per_portion_kcal")
).filter(
    col("calcium_per_portion").isNotNull()
).orderBy(desc("calcium_per_portion")).limit(5)
top_5_calcium.show(5, truncate=False)

print("\n" + "=" * 80)
print("TOP 5 FOODS BY IRON PER PORTION")
print("=" * 80)
top_5_iron = df_nutrients_per_portion.select(
    "food_description",
    "portion_description",
    spark_round(col("portion_grams"), 1).alias("portion_grams"),
    spark_round(col("iron_per_portion"), 2).alias("iron_per_portion_mg"),
    spark_round(col("energy_per_portion"), 1).alias("energy_per_portion_kcal")
).filter(
    col("iron_per_portion").isNotNull()
).orderBy(desc("iron_per_portion")).limit(5)
top_5_iron.show(5, truncate=False)


TOP 5 FOODS BY PROTEIN PER PORTION
+--------------------------------------------------------------------------------------------------------------+-------------------+-------------+---------------------+-----------------------+
|food_description                                                                                              |portion_description|portion_grams|protein_per_portion_g|energy_per_portion_kcal|
+--------------------------------------------------------------------------------------------------------------+-------------------+-------------+---------------------+-----------------------+
|Beef, round, eye of round roast, boneless, separable lean only, trimmed to 0" fat, all grades, cooked, roasted|1.0 undetermined   |346.0        |103.28               |2352.8                 |
|Beef, round, eye of round roast, boneless, separable lean only, trimmed to 0" fat, all grades, cooked, roasted|1.0 undetermined   |330.0        |98.51                |2244.0                 |
